In [9]:
# This script is for processing the dataset to form a new multi-task dataset with a unform labeling code
# Datasets PDBBind(Protein-Ligand, Protein-Protein), EnzymeCommission, GeneOntology
import os
import json
from typing import List
import random
import pandas as pd
from tqdm import tqdm

def cal_complex_all(json_dirs):
    complex_list = []
    for json_dir in json_dirs:
        with open(json_dir, 'r') as f:
            info_dict = json.load(f)
        for i in info_dict:
            if '-' in i:
                id = i.split('-')[0].lower()
            complex_list.append(id)
    print(len(list(set(complex_list))))
        

def gen_train_test_ids(complex_dict, ec_dict, go_dict):
    train_list = []
    # print(complex_dict)
    all_list = []
    for k, v in complex_dict.items():
        all_list.extend(v)
    all_list = list(set(all_list))
    for k, v in complex_dict.items():
        # print(k, v)
        # 由于每个蛋白可能有多个Uniprot，可以根据某一个Uniprot不在的情况找出训练集的样本
        if k not in ec_dict or k not in go_dict:
            train_list.extend(v)
    #取反即可获得标注完整的数据集
    test_list = [i for i in all_list if i not in train_list]
    test_uniprots = []
    for k, v in complex_dict.items():
        for id in v:
            if id in test_list and k not in test_uniprots:
                test_uniprots.append(k)
    # print("Uniprots:", len(test_uniprots))
    # print("Train:", len(train_list), len(list(set(train_list))))
    # print("Test:", len(test_list), len(list(set(test_list))))
    # print("All:", len(list(set(test_list + train_list))))
    # print(list(set(train_list)))
    return list(set(train_list)), list(set(test_list)), list(set(test_uniprots))
        
            
def gen_protein_property_uniprots(json_dir: str, single=True):
    with open(json_dir, 'r') as f:
        info_dict = json.load(f)
    uniprot_dict = {} # 蛋白质 dictionary
    print(len(info_dict))
    if '7NS4-b' in info_dict.keys():
        print('7NS4')
    info_dict_new = {}
    for k, v in info_dict.items():
        if single:
            if len(v) != 1:
                continue
            else:
                uniprot_id = v[0]
                info_dict_new[k] = [uniprot_id]   
                # 只记录同一个uniprot id的所有pdb_id
                if uniprot_id not in uniprot_dict:     
                    uniprot_dict[uniprot_id] = k
                # else:
                #     uniprot_dict[uniprot_id].append(k)
        else:
            # Remove items whose uniprot ids are larger than 3 or equal zero.
            if len(v) == 0 or len(v) > 3:
                continue
            else:
                info_dict_new[k] = v
                for uniprot_id in v:
                    if uniprot_id not in uniprot_dict:
                        uniprot_dict[uniprot_id] = [k]
                    else:
                        uniprot_dict[uniprot_id].append(k)
    # print("Unitest:", len(uniprot_dict), len(info_dict_new))
    return uniprot_dict, info_dict_new
    
def save_dataset_info(dir, complex_list):
    with open(dir, 'w') as f:
        for complex in complex_list:
            f.write(complex + '\n')
        f.close()

def gen_ec_labels():
    root_dir = './data/EnzymeCommission/nrPDB-EC_annot.tsv'
    # root_dir = './data/EC/nrPDB-EC_annot.tsv'
    download_pdb = './data/EC/download_pdb.json'
    with open(root_dir, 'r') as f:
        lines = f.readlines()
    with open(download_pdb, 'r') as d:
        download = json.load(d)
    ec_classes = lines[1].strip().split('\t')
    label_dict = {}
    pdb_annot_dict = {}
    label_id = 0
    for label in ec_classes:
        if label not in label_dict:
            label_dict[label] = label_id
            label_id += 1
    cnt = 0
    for item in lines[3:]:
        pdb_id, annotations = item.split('\t')
        annotations_list = annotations.strip().split(',')
        pdb_annot_dict[pdb_id] = [label_dict[annot] for annot in annotations_list]
    return pdb_annot_dict
    # print("Number of classes in task {} is {}".format('EnzymeCommission', label_id))

# def get_full_annotation(go_uniprot_dict):
#     root_dir = './datasets/GeneOntology/nrPDB-GO_annot.tsv'
#     with open(root_dir, 'r') as f:
#         lines = f.readlines()
#     go_classes_molecular_functions = lines[1].strip().split('\t')
#     go_classes_biological_process = lines[5].strip().split('\t')
#     go_classes_cellular_component = lines[9].strip().split('\t')
#     for k, v in go_uniprot_dict.items():
        

def gen_go_labels(go_uniprot_dict):
    root_dir = './data/GeneOntology/nrPDB-GO_annot.tsv'
    # root_dir = './data/GO/nrPDB-GO_annot.tsv'
    
    with open(root_dir, 'r') as f:
        lines = f.readlines()
    go_classes_molecular_functions = lines[1].strip().split('\t')
    go_classes_biological_process = lines[5].strip().split('\t')
    go_classes_cellular_component = lines[9].strip().split('\t')
    
    label_dict = {'molecular_functions':{}, 'biological_process':{}, 'cellular_component':{}}
    pdb_annot_dict = {}
    label_id_molecular = 0
    label_id_biological = 0
    label_id_cellular = 0
    full_ids = []
    full_uniprot_dict = {}
    for label in go_classes_molecular_functions:
        if label not in label_dict['molecular_functions']:
            label_dict['molecular_functions'][label] = label_id_molecular
            label_id_molecular += 1
    for label in go_classes_biological_process:
        if label not in label_dict['biological_process']:
            label_dict['biological_process'][label] = label_id_biological
            label_id_biological += 1
    for label in go_classes_cellular_component:
        if label not in label_dict['cellular_component']:
            label_dict['cellular_component'][label] = label_id_cellular
            label_id_cellular += 1
    for item in tqdm(lines[13:]):
        pdb_id, molecular, biological, cellular = item.split('\t')
        flag = False
        # for go_uniprot_values in go_uniprot_dict.values():
        #     if pdb_id in go_uniprot_values:
        #         flag = True
        #         break
        if pdb_id in go_uniprot_dict.values():
            if pdb_id == '3EWD-A':
                print(molecular, biological, cellular)
            molecular_list  = molecular.strip().split(',')
            biological_list = biological.strip().split(',')
            cellular_list = cellular.strip().split(',')
            # print(molecular_list)
            # 列表中会包含一些空的信息
            pdb_annot_dict[pdb_id] = {'molecular_functions':[label_dict['molecular_functions'][annot] for annot in molecular_list if annot != ''],
                                    'biological_process':[label_dict['biological_process'][annot] for annot in biological_list if annot != ''],
                                    'cellular_component':[label_dict['cellular_component'][annot] for annot in cellular_list if annot != '']}
            if pdb_annot_dict[pdb_id]['molecular_functions'] != [] and pdb_annot_dict[pdb_id]['biological_process'] != [] and pdb_annot_dict[pdb_id]['cellular_component'] != []:
                full_ids.append(pdb_id)
            if pdb_id == '6A12-A' or pdb_id == '3EWD-A':
                print(pdb_annot_dict[pdb_id])
    for k, v in go_uniprot_dict.items():
        if v in full_ids:
            full_uniprot_dict[k] = v
    print("Number of classes in task {} is {}".format('molecular_functions', label_id_molecular+1))
    print("Number of classes in task {} is {}".format('biological_process', label_id_biological+1))
    print("Number of classes in task {} is {}".format('cellular_component', label_id_cellular+1))
    # print(pdb_annot_dict)
    return pdb_annot_dict, full_uniprot_dict
    # print(pdb_annot_dict)


def gen_lba_labels():
    root_dir = './data/PDBbind/refined-set/index/INDEX_general_PL_data.2020'
    res = {}
    with open(root_dir) as f:
        for line in f:
            if '#' in line:
                continue
            cont = line.strip().split()
            if len(cont) < 5:
                continue
            code, pk = cont[0], cont[3]
            res[code] = float(pk)
    # print("LBA:", len(res))
    return res

def gen_ppi_labels():
    root_dir = './data/protein_protein/pp_affinity.xlsx'
    pp_info = pd.read_excel(root_dir, header=1)
    pdb_codes = pp_info['PDB code']
    res = {}
    for i, code in enumerate(pdb_codes):
        res[code] = pp_info['pKd pKi pIC50'][i]
    # print("PPI:", len(res))
    return res

def gen_label(pdb_ids, ec_labels, go_labels, ppi_labels, lba_labels, ec_uniprot_dict, go_uniprot_dict, ec_info_dict, go_info_dict, pp_info_dict, pl_info_dict):
    uniform_labels = {}
    
    for pdb_id in pdb_ids:
        try:
            if pdb_id[:4] == '7NS4':
                print(pdb_id)
            if '-' in pdb_id:
                if pdb_id == '6A12-A':
                    print('6A12-A')
                    print(go_info_dict[pdb_id])
                    # print(go_labels)
                    print('6A12-A')
                if pdb_id in ec_labels:
                    uniprots = ec_info_dict[pdb_id]
                    single_ec_label = [ec_labels[pdb_id]]
                else:
                    single_ec_label = [-1]
                if pdb_id in go_labels:
                    if pdb_id == '6A12-A':
                        print("go 6A12-A")
                    uniprots = go_info_dict[pdb_id]
                    single_go_label = [go_labels[pdb_id]]
                else:
                    single_go_label = [-1]
            else:
                single_ec_label = []
                single_go_label = []
                if pdb_id in pp_info_dict:
                    uniprots = pp_info_dict[pdb_id]
                elif pdb_id in pl_info_dict:
                    uniprots = pl_info_dict[pdb_id]
                else:
                    raise NotImplementedError
                for uniprot_id in uniprots:
                    if uniprot_id in ec_uniprot_dict:
                        ec_label = ec_labels[ec_uniprot_dict[uniprot_id]]
                    else:
                        ec_label = -1
                    if uniprot_id in go_uniprot_dict:
                        go_label = go_labels[go_uniprot_dict[uniprot_id]]
                    else:
                        go_label = -1
                    single_ec_label.append(ec_label)
                    single_go_label.append(go_label)
            if pdb_id in ppi_labels:
                ppi_label = ppi_labels[pdb_id]
            else:
                ppi_label = -1
            if pdb_id in lba_labels:
                lba_label = lba_labels[pdb_id]
            else:
                lba_label = -1
            uniform_labels[pdb_id] = {"uniprots":uniprots, "ec": single_ec_label, "go": single_go_label, "ppi": ppi_label, "lba": lba_label}
        except Exception as exp:
            pass
            # print(exp)
            # print('pdbid:', pdb_id)
    return uniform_labels
    # print(uniform_labels)

In [2]:
def gen_reaction_labels():
    root_dir = './data/ProtFunc/chain_functions.txt'
    pdb_uniprot_dict = json.load(open('./output_info/reaction_uniprots.json'))
    f = open(root_dir)
    lines = f.readlines()
    pdb_annot_dict = {}
    for line in lines:
        pdbid,label = line.split(',')
        pdbid = pdbid.replace('.','-').upper()
        if(pdbid in pdb_uniprot_dict.keys()):
            uniprot_ids = pdb_uniprot_dict[pdbid]
            pdb_annot_dict[pdbid] = [int(label.strip('\n'))]
    return pdb_annot_dict

def gen_reaction_uniprots():
    "生成uniprot_dict, info_dict"
    input_json_dir = "./output_info/enzyme_commission_new_uniprots.json"
    output_json_dir = "./output_info/reaction_uniprots.json"
    all_uniprots_dict = json.load(open(input_json_dir))
    new_uniprots_dict = {}
    chain_function_file = open('./data/ProtFunc/chain_functions.txt','r')
    for line in chain_function_file.readlines():
        pdbid = line.split(',')[0]
        pdbid = pdbid.replace('.','-').upper()
        try:
            new_uniprots_dict[pdbid] = all_uniprots_dict[pdbid]
        except:
            print("{} not found".format(pdbid))
    json.dump(new_uniprots_dict,open(output_json_dir,'w'))
    return gen_protein_property_uniprots(output_json_dir)

In [4]:
def gen_train_test_ids_new(complex_dict,ec_dict,go_dict,reaction_dict):
    train_list = []
    # print(complex_dict)
    all_list = []
    for k, v in complex_dict.items():
        all_list.extend(v) # k is uniprot id v is pdb ids
    all_list = list(set(all_list))
    for k, v in complex_dict.items():
        # print(k, v)
        # 由于每个蛋白可能有多个Uniprot，可以根据某一个Uniprot不在的情况找出训练集的样本
        if k not in ec_dict or k not in go_dict or k not in reaction_dict:
            train_list.extend(v) # train_list:哪些pdb_id中有uniprot缺少ec或者go标注
    #取反即可获得标注完整的数据集
    test_list = [i for i in all_list if i not in train_list] # test_list:哪些pdb_id中所有uniprot有完整的ec和go标注
    test_uniprots = []
    for k, v in complex_dict.items():
        for id in v:
            if id in test_list and k not in test_uniprots:
                test_uniprots.append(k)
    # print("Uniprots:", len(test_uniprots))
    # print("Train:", len(train_list), len(list(set(train_list))))
    # print("Test:", len(test_list), len(list(set(test_list))))
    # print("All:", len(list(set(test_list + train_list))))
    # print(list(set(train_list)))
    return list(set(train_list)), list(set(test_list)), list(set(test_uniprots))

def gen_label_new(pdb_ids, ec_labels, go_labels,reaction_labels, ppi_labels, lba_labels, ec_uniprot_dict, go_uniprot_dict,reaction_uniprot_dict, ec_info_dict, go_info_dict,reaction_info_dict, pp_info_dict, pl_info_dict):
    uniform_labels = {}
    for pdb_id in pdb_ids:
        if '-' in pdb_id:
            if pdb_id in ec_labels:
                uniprots = ec_info_dict[pdb_id]
                single_ec_label = [ec_labels[pdb_id]]
            else:
                single_ec_label = [-1]
            if pdb_id in go_labels:
                uniprots = go_info_dict[pdb_id]
                single_go_label = [go_labels[pdb_id]]
            else:
                single_go_label = [-1]
            if pdb_id in reaction_labels and pdb_id in reaction_info_dict:
                uniprots = reaction_info_dict[pdb_id] 
                single_reaction_label = [reaction_labels[pdb_id]]
            else:
                single_reaction_label = [-1]
                
        else:
            single_ec_label = []
            single_go_label = []
            single_reaction_label = []
            if pdb_id in pp_info_dict:
                uniprots = pp_info_dict[pdb_id]
            elif pdb_id in pl_info_dict:
                uniprots = pl_info_dict[pdb_id]
            else:
                raise NotImplementedError
            for uniprot_id in uniprots:
                if uniprot_id == '1A5H-A':
                    print(uniprots, pdb_id)
                if uniprot_id in ec_uniprot_dict:
                    ec_label = ec_labels[ec_uniprot_dict[uniprot_id]]
                else:
                    ec_label = -1
                if uniprot_id in go_uniprot_dict:
                    go_label = go_labels[go_uniprot_dict[uniprot_id]]
                else:
                    go_label = -1
                if uniprot_id in reaction_uniprot_dict:
                    reaction_label = reaction_labels[reaction_uniprot_dict[uniprot_id]]
                else:
                    reaction_label = -1
                single_ec_label.append(ec_label)
                single_go_label.append(go_label)
                single_reaction_label.append(reaction_label)
        if pdb_id in ppi_labels:
            ppi_label = ppi_labels[pdb_id]
        else:
            ppi_label = -1
        if pdb_id in lba_labels:
            lba_label = lba_labels[pdb_id]
        else:
            lba_label = -1
        uniform_labels[pdb_id] = {"uniprots":uniprots, "ec": single_ec_label, "go": single_go_label, "reaction":single_reaction_label,"ppi": ppi_label, "lba": lba_label}
    return uniform_labels

In [36]:
def gen_train_test_ids_fold(complex_dict, ec_dict, go_dict, reaction_dict, fold_dict):
    train_list = []
    # print(complex_dict)
    all_list = []
    for k, v in complex_dict.items():
        all_list.extend(v) # k is uniprot id v is pdb ids
    all_list = list(set(all_list))
    for k, v in complex_dict.items():
        # print(k, v)
        # 由于每个蛋白可能有多个Uniprot，可以根据某一个Uniprot不在的情况找出训练集的样本
        if k not in ec_dict or k not in go_dict or k not in reaction_dict or k not in fold_dict:
            train_list.extend(v) # train_list:哪些pdb_id中有uniprot缺少ec或者go标注
    #取反即可获得标注完整的数据集
    test_list = [i for i in all_list if i not in train_list] # test_list:哪些pdb_id中所有uniprot有完整的ec和go标注
    test_uniprots = []
    for k, v in complex_dict.items():
        for id in v:
            if id in test_list and k not in test_uniprots:
                test_uniprots.append(k)
    # print("Uniprots:", len(test_uniprots))
    # print("Train:", len(train_list), len(list(set(train_list))))
    # print("Test:", len(test_list), len(list(set(test_list))))
    # print("All:", len(list(set(test_list + train_list))))
    # print(list(set(train_list)))
    return list(set(train_list)), list(set(test_list)), list(set(test_uniprots))

def gen_label_fold(pdb_ids, ec_labels, go_labels, reaction_labels, fold_labels,
                   ppi_labels, lba_labels,
                   ec_uniprot_dict, go_uniprot_dict,reaction_uniprot_dict, fold_uniprot_dict,
                   ec_info_dict, go_info_dict, reaction_info_dict, fold_info_dict,
                   pp_info_dict, pl_info_dict):
    uniform_labels = {}
    for pdb_id in pdb_ids:
        if '-' in pdb_id:
            if pdb_id in ec_labels:
                uniprots = ec_info_dict[pdb_id]
                single_ec_label = [ec_labels[pdb_id]]
            else:
                single_ec_label = [-1]
            if pdb_id in go_labels:
                uniprots = go_info_dict[pdb_id]
                single_go_label = [go_labels[pdb_id]]
            else:
                single_go_label = [-1]
            if pdb_id in reaction_labels and pdb_id in reaction_info_dict:
                uniprots = reaction_info_dict[pdb_id] 
                single_reaction_label = [reaction_labels[pdb_id]]
            else:
                single_reaction_label = [-1]
            if pdb_id in fold_labels and pdb_id in fold_info_dict:
                uniprots = fold_info_dict[pdb_id] 
                single_fold_label = [fold_labels[pdb_id]]
            else:
                single_fold_label = [-1]
        else:
            single_ec_label = []
            single_go_label = []
            single_reaction_label = []
            single_fold_label = []
            if pdb_id in pp_info_dict:
                uniprots = pp_info_dict[pdb_id]
            elif pdb_id in pl_info_dict:
                uniprots = pl_info_dict[pdb_id]
            else:
                raise NotImplementedError
            for uniprot_id in uniprots:
                if uniprot_id == '1A5H-A':
                    print(uniprots, pdb_id)
                if uniprot_id in ec_uniprot_dict:
                    ec_label = ec_labels[ec_uniprot_dict[uniprot_id]]
                else:
                    ec_label = -1
                if uniprot_id in go_uniprot_dict:
                    go_label = go_labels[go_uniprot_dict[uniprot_id]]
                else:
                    go_label = -1
                if uniprot_id in reaction_uniprot_dict:
                    reaction_label = reaction_labels[reaction_uniprot_dict[uniprot_id]]
                else:
                    reaction_label = -1
                if uniprot_id in fold_uniprot_dict:
                    fold_label = fold_labels[fold_uniprot_dict[uniprot_id]]
                else:
                    fold_label = -1
                single_ec_label.append(ec_label)
                single_go_label.append(go_label)
                single_reaction_label.append(reaction_label)
                single_fold_label.append(fold_label)
        if pdb_id in ppi_labels:
            ppi_label = ppi_labels[pdb_id]
        else:
            ppi_label = -1
        if pdb_id in lba_labels:
            lba_label = lba_labels[pdb_id]
        else:
            lba_label = -1
        uniform_labels[pdb_id] = {"uniprots":uniprots, "ec": single_ec_label, "go": single_go_label, "reaction":single_reaction_label, "fold": single_fold_label,"ppi": ppi_label, "lba": lba_label}
    return uniform_labels

In [27]:
def gen_fold_labels():
    root_dir = './data/HomologyTAPE/nrPDB-fold_annot.tsv'
    class_map_dir = './data/HomologyTAPE/class_map.txt'
    with open(root_dir, 'r') as f:
        lines = f.readlines()
    class_map = {}
    with open('./data/HomologyTAPE/class_map.txt') as f:
        class_lines = f.readlines()
    for line in class_lines:
        # print(line)
        data = line[:-1].split('\t')
        # print(data)
        data = [d.strip() for d in data]
        class_map[data[0]] = int(data[1])
    pdb_annot_dict = {}
    for item in lines:
        # print(item)
        pdb_id, annotations = item[:-1].split('\t')
        annotations_list = annotations.strip().split(',')
        # print(annotations)
        pdb_annot_dict[pdb_id] = [class_map[annot] for annot in annotations_list]
    return pdb_annot_dict

In [10]:

print("Start processing original datasets to a multitask dataset")
root_dir = './output_info/'
json_files = ['enzyme_commission_uniprots.json', 'gene_ontology_uniprots.json', 'protein_protein_uniprots.json', 'protein_ligand_uniprots.json', 'fold_uniprots.json']
# json_files = ['ec_uniprots.json', 'gene_ontology_uniprots.json', 'protein_protein_uniprots.json', 'protein_ligand_uniprots.json']
# json_files = ['ec_uniprots.json', 'go_uniprots.json', 'protein_protein_uniprots.json', 'protein_ligand_uniprots.json', 'fold_uniprots.json']
json_dirs = [os.path.join(root_dir, json_file) for json_file in json_files]
# cal_complex_all(json_dirs)
# uniprot_dict: {Uniprot id: [List of pdb ids]}
# info_dict: {PDB id: [List of uniprot ids]}
ec_uniprot_dict, ec_info_dict = gen_protein_property_uniprots(json_dirs[0])
if '7OVA-AAA' in ec_info_dict.keys() or '7OVA' in ec_info_dict.keys():
    print("7OVA-AAA")
go_uniprot_dict, go_info_dict = gen_protein_property_uniprots(json_dirs[1])
pp_uniprot_dict, pp_info_dict = gen_protein_property_uniprots(json_dirs[2], single=False)
pl_uniprot_dict, pl_info_dict = gen_protein_property_uniprots(json_dirs[3], single=False)
fold_uniprot_dict, fold_info_dict = gen_protein_property_uniprots(json_dirs[4])
print("Number of samples pl, pp, ec, go:", len(pl_info_dict), len(pp_info_dict), len(ec_info_dict), len(go_info_dict))
ec_labels = gen_ec_labels()
go_labels, go_full_uniprot_dict = gen_go_labels(go_uniprot_dict)
ppi_labels = gen_ppi_labels()
lba_labels = gen_lba_labels()
print(len(ec_uniprot_dict), len(go_uniprot_dict), len(pp_uniprot_dict), len(pl_uniprot_dict))

# full_train_ratio = 0.2
# full_val_ratio = 0.4
# full_test_ratio = 0.4


Start processing original datasets to a multitask dataset
19199
36641
2854
5318
10949
Number of samples pl, pp, ec, go: 5208 2662 18810 34944


 39%|███▉      | 14470/36641 [00:30<00:30, 733.53it/s]

GO:0019239 GO:0034654,GO:0009161,GO:0009167,GO:0009126,GO:0006793,GO:0055086,GO:0009156,GO:0009127,GO:0019637,GO:0009123,GO:0009168,GO:0090407,GO:0009124,GO:0006753,GO:1901293,GO:0006796,GO:0019438 

{'molecular_functions': [59], 'biological_process': [767, 1735, 619, 1497, 345, 309, 447, 849, 1506, 1685, 1220, 1247, 694, 240, 488, 956, 546], 'cellular_component': []}


100%|██████████| 36641/36641 [01:15<00:00, 486.61it/s] 


Number of classes in task molecular_functions is 490
Number of classes in task biological_process is 1944
Number of classes in task cellular_component is 321
15688 29732 2695 1590


In [28]:
fold_labels = gen_fold_labels()

In [19]:
for key, value in pl_info_dict.items():
    if '1A5H-A' in value:
        print('ok')

In [30]:
reaction_uniprot_dict,reaction_info_dict = gen_reaction_uniprots()
reaction_labels = gen_reaction_labels()
# print(len(ec_uniprot_dict), len(go_uniprot_dict), len(pp_uniprot_dict), len(pl_uniprot_dict))
train_list_pp, test_list_pp, test_uniprots_1 = gen_train_test_ids_fold(pp_uniprot_dict, ec_uniprot_dict, go_full_uniprot_dict,reaction_dict=reaction_uniprot_dict, fold_dict=fold_uniprot_dict)
train_list_pl, test_list_pl, test_uniprots_2 = gen_train_test_ids_fold(pl_uniprot_dict, ec_uniprot_dict, go_full_uniprot_dict,reaction_dict=reaction_uniprot_dict, fold_dict=fold_uniprot_dict)
test_list_all = list(set(test_list_pl + test_list_pp))
test_uniprots_all = list(set(test_uniprots_1 + test_uniprots_2))

# full_train_ratio = 0.2
# full_val_ratio = 0.4
# full_test_ratio = 0.4
random.shuffle(test_list_pl)
random.shuffle(test_list_pp)

full_test_list = test_list_pl[int(0.6*len(test_list_pl)):] + test_list_pp[int(0.6*len(test_list_pp)):]
full_val_list = test_list_pl[int(0.2*len(test_list_pl)): int(0.6*(len(test_list_pl)))] + test_list_pp[int(0.2*len(test_list_pp)): int(0.6*len(test_list_pp))]
full_train_list = test_list_pl[: int(0.2*len(test_list_pl))] + test_list_pp[: int(0.2*len(test_list_pp))]

train_list_ec = [ec_uniprot_dict[i] for i in ec_uniprot_dict if i not in test_uniprots_all]
train_list_go = [go_uniprot_dict[i] for i in go_uniprot_dict if i not in test_uniprots_all]
train_list_reaction = [reaction_uniprot_dict[i] for i in reaction_uniprot_dict if i not in test_uniprots_all]
train_list_fold = [fold_uniprot_dict[i] for i in fold_uniprot_dict if i not in test_uniprots_all]

# train_list_pl:有pl label但是ec或者go缺点啥的pdb id训练集
# train_list_pp:有pp label但是ec或者go缺点啥的训练集
# train_list_ec:有ec但是没有go的训练集
# train_list_go:有go但是没有ec的训练集
train_list_all = list(set(train_list_pl + train_list_pp + train_list_ec + train_list_go +train_list_reaction+ full_train_list))
print("Train List:", len(train_list_ec), len(train_list_go),len(train_list_reaction),len(train_list_all))
# train/val/test.txt contains samples of full labels, while train_all.txt contains labals with partial labels and samples in train.txt.
print("Process finished, saving information into ./dataset/MultiTask_new/")
if os.path.exists('./datasets/MultiTask_new/train_all.txt') and os.path.exists('./datasets/MultiTask/tmp/train.txt'):
    print("File already exists, skip saving split information...")
else:
    print("Saving split information...")
    os.makedirs('./datasets/MultiTask_new', exist_ok=True)
    save_dataset_info('./datasets/MultiTask_new/train_all.txt', train_list_all)
    save_dataset_info('./datasets/MultiTask_new/train.txt', full_train_list)
    save_dataset_info('./datasets/MultiTask_new/val.txt', full_val_list)
    save_dataset_info('./datasets/MultiTask_new/test.txt', full_test_list)

# uniformed_label_dict = gen_label(train_list_all+full_test_list+full_val_list, ec_labels, go_labels, ppi_labels, lba_labels, ec_uniprot_dict, go_uniprot_dict, ec_info_dict, go_info_dict, pp_info_dict, pl_info_dict)    


37400
Train List: 15613 29657 6718 45165
Process finished, saving information into ./dataset/MultiTask_new/
Saving split information...


In [35]:
ec_labels

{'4PR3-A': [156, 286],
 '1TNT-A': [421, 22],
 '1T8A-A': [324, 457],
 '5H75-A': [253],
 '2FOR-A': [256, 124],
 '1Z7L-A': [140],
 '4XL3-A': [334],
 '2V4J-B': [54],
 '4AZV-A': [65, 103],
 '4HTO-A': [421, 325],
 '4G2C-A': [441, 24],
 '4H2H-A': [21],
 '1ZXA-A': [500, 67],
 '6P5W-A': [324, 457],
 '5KB6-A': [103],
 '6GFW-M': [191, 335],
 '1YQQ-A': [416, 12],
 '1E5F-A': [487],
 '1MHZ-G': [518, 146],
 '4UY5-A': [65],
 '1Z19-A': [335],
 '2K31-A': [238, 210],
 '2CU1-A': [301, 67],
 '4HN0-A': [119],
 '4Y4R-A': [78, 217],
 '2WMY-A': [495, 180],
 '3K13-A': [65],
 '4HE8-A': [434],
 '3SN2-A': [322],
 '3CQL-A': [294, 457],
 '4JK2-E': [191, 335],
 '1R8U-B': [196, 6],
 '1S7G-A': [196, 187],
 '3QWZ-A': [507, 139],
 '5GLH-A': [324, 457],
 '3EZB-A': [449, 111],
 '5N6M-A': [196],
 '6W80-A': [336, 420],
 '4OJY-A': [517, 457],
 '6FHW-A': [297, 457],
 '5HBZ-A': [9, 139, 335, 45, 69, 30],
 '4KQB-A': [67, 47],
 '4P0Q-A': [22],
 '6A47-A': [306, 85],
 '5UWZ-A': [43],
 '5UJC-A': [476],
 '2BAY-A': [345, 207],
 '3P5W-

In [37]:
all_list = train_list_all+full_test_list+full_val_list
uniformed_label_dict = gen_label_fold(all_list,ec_labels, go_labels,reaction_labels,fold_labels,
                                      ppi_labels, lba_labels,
                                      ec_uniprot_dict, go_uniprot_dict,reaction_uniprot_dict,fold_uniprot_dict,
                                      ec_info_dict,go_info_dict,reaction_info_dict,fold_info_dict,
                                      pp_info_dict,pl_info_dict)
print("An example of the processed unified label:\n", full_test_list[0], ": ", uniformed_label_dict[full_test_list[0]])
if os.path.exists('./datasets/MultiTask_new/uniformed_labels.json'):
    print("File already exists, skip saving uniformed labels...")
else:
    print('Generating uniformed labels...')
    with open('./datasets/MultiTask_new/uniformed_labels.json', 'w') as f:
        json.dump(uniformed_label_dict, f)

An example of the processed unified label:
 1qbn :  {'uniprots': ['P00760'], 'ec': [[23, 27]], 'go': [{'molecular_functions': [386, 36, 243, 383, 192, 47, 475, 35, 382, 81], 'biological_process': [57, 806, 299], 'cellular_component': [195, 81]}], 'reaction': [[208]], 'fold': [[328]], 'ppi': -1, 'lba': 5.85}
Generating uniformed labels...


In [26]:
ec_labels = gen_ec_labels()
ec_labels['1A5H-A']

[1210]

In [21]:
go_task_dic = {}
for k, v in go_uniprot_dict.items():
    go_task_dic[v] = [k]
print(len(go_task_dic))

32554


In [22]:
with open('./output_info/go_task_uniprots.json', 'w') as f:
    json.dump(go_task_dic, f)

In [23]:
go_uniprot_dict['A5KE01']

'2PGF-A'

In [3]:
random.shuffle(test_list_pl)
random.shuffle(test_list_pp)
print(len(test_list_pl), len(test_list_pp))
full_test_list = test_list_pl[int(0.6*len(test_list_pl)):] + test_list_pp[int(0.6*len(test_list_pp)):]
full_val_list = test_list_pl[int(0.2*len(test_list_pl)): int(0.6*(len(test_list_pl)))] + test_list_pp[int(0.2*len(test_list_pp)): int(0.6*len(test_list_pp))]
full_train_list = test_list_pl[: int(0.2*len(test_list_pl))] + test_list_pp[: int(0.2*len(test_list_pp))]

train_list_ec = [ec_uniprot_dict[i] for i in ec_uniprot_dict if i not in test_uniprots_all]
train_list_go = [go_uniprot_dict[i] for i in go_uniprot_dict if i not in test_uniprots_all]
print("Train List:", len(train_list_ec), len(train_list_go))
train_list_all = list(set(train_list_pl + train_list_pp + train_list_ec + train_list_go + full_train_list))

# train/val/test.txt contains samples of full labels, while train_all.txt contains labals with partial labels and samples in train.txt.
print("Process finished, saving information into ./dataset/MultiTask_go/")
if os.path.exists('./datasets/MultiTask_go/train_all.txt') and os.path.exists('./datasets/MultiTask_go/tmp/train.txt'):
    print("File already exists, skip saving split information...")
else:
    print("Saving split information...")
    os.makedirs('./datasets/MultiTask_go', exist_ok=True)
    save_dataset_info('./datasets/MultiTask_go/train_all.txt', train_list_all)
    save_dataset_info('./datasets/MultiTask_go/train.txt', full_train_list)
    save_dataset_info('./datasets/MultiTask_go/val.txt', full_val_list)
    save_dataset_info('./datasets/MultiTask_go/test.txt', full_test_list)
    
# print(go_labels)
all_list = train_list_all+full_test_list+full_val_list
if '6A12-A' in all_list:
    print("ok")
uniformed_label_dict = gen_label(train_list_all+full_test_list+full_val_list, ec_labels, go_labels, ppi_labels, lba_labels, ec_uniprot_dict, go_uniprot_dict, ec_info_dict, go_info_dict, pp_info_dict, pl_info_dict)    
print("An example of the processed unified label:\n", full_test_list[0], ": ", uniformed_label_dict[full_test_list[0]])
if os.path.exists('./datasets/MultiTask_go/uniformed_labels.json'):
    print("File already exists, skip saving uniformed labels...")
else:
    print('Generating uniformed labels...')
    with open('./datasets/MultiTask_go/uniformed_labels.json', 'w') as f:
        json.dump(uniformed_label_dict, f)

1548 122


Train List: 15267 32133
Process finished, saving information into ./dataset/MultiTask_go/
Saving split information...
ok
7NS4-b
6A12-A
['Q8L1V2']
6A12-A
go 6A12-A
7NS4-i
An example of the processed unified label:
 2zdm :  {'uniprots': ['P00760'], 'ec': [[23, 27]], 'go': [{'molecular_functions': [3913, 4714, 4014, 3938, 3247, 459, 1269, 1849], 'biological_process': [4961, 4977], 'cellular_component': [343, 986, 1466]}], 'ppi': -1, 'lba': 6.41}
File already exists, skip saving uniformed labels...


In [4]:
all_list = train_list_all+full_test_list+full_val_list
if '101M-A' in train_list_go:
    print("ok")

ok


In [44]:
for key, value in tqdm(uniformed_label_dict.items()):
    if '-' in key and len(value['uniprots']) > 1:
        print(key)

  0%|          | 0/48217 [00:00<?, ?it/s]

100%|██████████| 48217/48217 [00:00<00:00, 359480.88it/s]


In [38]:
go_annot_dir = './data/GO/nrPDB-GO_annot.tsv'
with open(go_annot_dir, 'r') as f:
    go_annots = f.readlines()
label_dict = {'molecular_functions':{}, 'biological_process':{}, 'cellular_component':{}}
pdb_annot_dict = {}
label_id_molecular = 0
label_id_biological = 0
label_id_cellular = 0
full_ids = []
full_uniprot_dict = {}
go_classes_molecular_functions = go_annots[1].strip().split('\t')
go_classes_biological_process = go_annots[5].strip().split('\t')
go_classes_cellular_component = go_annots[9].strip().split('\t')
for label in go_classes_molecular_functions:
    if label not in label_dict['molecular_functions']:
        label_dict['molecular_functions'][label] = label_id_molecular
        label_id_molecular += 1
for label in go_classes_biological_process:
    if label not in label_dict['biological_process']:
        label_dict['biological_process'][label] = label_id_biological
        label_id_biological += 1
for label in go_classes_cellular_component:
    if label not in label_dict['cellular_component']:
        label_dict['cellular_component'][label] = label_id_cellular
        label_id_cellular += 1
for annot in go_annots[12:]:
    pdb_id, mf, bp, cc = annot.split('\t')
    if pdb_id[:4] == '101M':
        print(pdb_id)
    if pdb_id in uniformed_label_dict.keys():
        molecular_list  = mf.strip().split(',')
        biological_list = bp.strip().split(',')
        cellular_list = cc.strip().split(',')
        # print(molecular_list)
        # 列表中会包含一些空的信息
        uniformed_label_dict[pdb_id]['go'] = [{'molecular_functions':[label_dict['molecular_functions'][annot] for annot in molecular_list if annot != ''],
                                'biological_process':[label_dict['biological_process'][annot] for annot in biological_list if annot != ''],
                                'cellular_component':[label_dict['cellular_component'][annot] for annot in cellular_list if annot != '']}]
    

101M-A


In [9]:
with open('./datasets/MultiTask_new/uniformed_labels.json', 'w') as f:
    json.dump(uniformed_label_dict, f)

In [8]:
uniformed_label_dict['101M-A']

{'uniprots': ['P18946'],
 'ec': [-1],
 'go': [{'molecular_functions': [4714, 4354],
   'biological_process': [6431, 7623],
   'cellular_component': []}],
 'ppi': -1,
 'lba': -1}

In [35]:
train_list_pl[0]

'2qw1'

In [27]:
if '101M-A' in train_list_ec:
    print("yes")

yes
